# Traffic Analysis and Prediction Model with YOLOv8

This notebook sets up a traffic analysis and prediction model using YOLOv8. It will:

1. Load the dataset of traffic images.
2. Train the YOLOv8 model.
3. Detect cars in a continuous video stream from a CCTV camera.
4. Show the density of cars.
5. Continuously train the model with new data.


In [ ]:
# Install necessary packages
!pip install torch torchvision torchaudio
!pip install opencv-python
!pip install scikit-learn
!pip install ultralytics

## Import Libraries

In [ ]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import precision_score, recall_score, f1_score
import os
import time

## Load and Prepare Dataset

In [ ]:
# Define paths
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Function to copy dataset files (Assuming dataset is already prepared)
def copy_dataset(source_dir, target_dir):
    for filename in os.listdir(source_dir):
        file_path = os.path.join(source_dir, filename)
        if os.path.isfile(file_path):
            shutil.copy(file_path, target_dir)

# Copy dataset files (replace 'path_to_your_dataset' with the actual path)
copy_dataset('path_to_your_dataset/train', train_dir)
copy_dataset('path_to_your_dataset/val', val_dir)
copy_dataset('path_to_your_dataset/test', test_dir)

## Train YOLOv8 Model

In [ ]:
# Define model configuration and hyperparameters
model_config = 'models/yolov8.yaml'

# Initialize model
yolo_model = YOLO(model_config)

# Train the model
yolo_model.train(data=train_dir, val=val_dir, epochs=50, batch_size=16)

# Save the trained model
yolo_model.save('models/yolov8_trained.pth')

## Detect Cars in Continuous Video Stream and Show Density

In [ ]:
# Function to calculate car density
def calculate_density(results, frame_shape):
    car_count = len(results)
    frame_area = frame_shape[0] * frame_shape[1]
    density = car_count / frame_area
    return density

# Load the trained model
yolo_model.load('models/yolov8_trained.pth')

# Open video stream from CCTV camera (replace '0' with the actual camera URL if needed)
capture = cv2.VideoCapture(0)

while True:
    ret, frame = capture.read()
    if not ret:
        break
    
    # Perform detection
    results = yolo_model.predict(frame)
    
    # Draw bounding boxes and calculate density
    density = calculate_density(results, frame.shape)
    for result in results:
        x1, y1, x2, y2, conf, cls = result
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Display density on frame
    cv2.putText(frame, f'Density: {density:.4f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Show frame
    cv2.imshow('Traffic Detection', frame)
    
    # Press 'q' to quit
da    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

## Continuously Train Model with New Data

In [ ]:
# Function to continuously train the model with new data
def continuous_training(model, new_data_dir, epochs=10):
    model.train(data=new_data_dir, epochs=epochs, batch_size=16)
    model.save('models/yolov8_continuously_trained.pth')

# Example usage (replace 'path_to_new_data' with the actual path to new data)
continuous_training(yolo_model, 'path_to_new_data')

## Evaluate Model

In [ ]:
# Function to evaluate the model
def evaluate_model(model, test_data_dir):
    y_true = []
    y_pred = []
    for image_path, labels in test_data_dir:
        image = cv2.imread(image_path)
        results = model.predict(image)
        
        y_true.extend(labels)
        y_pred.extend([result[5] for result in results])
    
    # Calculate metrics
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')

# Evaluate the model
evaluate_model(yolo_model, test_dir)